In [1]:
import os
import sys
import math
import json

import numpy as np
import cadquery as cq
from jupyter_cadquery import show
import matplotlib.pyplot as plt
%matplotlib inline

from cquav.wing.airfoil import Airfoil, load_airfoils_collection
from cquav.wing.profile import AirfoilSection, ThreeChamberBoxedWingSection
from cquav.wing.rect_console import RectangularWingConsole
from cquav.materials import IsotropicMaterial, FluidProperties

Overwriting auto display for cadquery Workplane and Shape


## Load airfoils collection

In [2]:
airfoils_collection = load_airfoils_collection()
airfoil_data = airfoils_collection["NACA 4 digit airfoils"]["NACA 2412 (naca2412-il)"]
airfoil = Airfoil(airfoil_data)

## Define constants

In [3]:
velocity_max = 35 #[m/s]
air_density = 1.225 #[kg/m^3]
kinematic_viscosity = 1.460*1e-5 ## [m^2/s]
dyn_airpressure_max = 0.5 * air_density * velocity_max**2 ## dynamic pressure
delta_max = 0.1 ## relative wing tip displacement
payload_mass = 10 ## kg
g = 9.81 ## [m/s**2]
load_factor = 1.2
cd_payload = 0.5
payload_cross_section_area = math.pi*0.1**2 / 4

## Materials

In [4]:
XPS_foam = IsotropicMaterial(30, 1e3, 25*1e6)
# https://www.mdpi.com/2073-4360/12/1/47

PETG = IsotropicMaterial(1270, 45*1e6, 2.1*1e9)
# https://www.researchgate.net/publication/362844605_A_Comparative_Study_for_Material_Selection_in_3D_Printing_of_Scoliosis_Back_Brace

Fiberglass_laminate = IsotropicMaterial(1800, 290*1e6, 12.4*1e9)
#https://laminatedplastics.com/fiberglasslaminates.pdf

Carbon_laminate = IsotropicMaterial(1500, 450*1e6, 35*1e9)
# https://www.researchgate.net/publication/259461841_Mechanical_Properties_of_Carbon_FiberEpoxy_Composites_Effects_of_Number_of_Plies_Fiber_Contents_and_Angle-Ply_Layers

In [5]:
air_props = FluidProperties(air_density, velocity_max, kinematic_viscosity)

## Build reinforced wing console

In [6]:
## Fixed parameters, mm
console_chord = 260
console_length = 900

max_chord_length = 1000
min_chord_length = 50

max_console_length = 12000
min_console_length = 50

MATERIALS = {"box": Fiberglass_laminate, "shell": Fiberglass_laminate, "foam": XPS_foam}

In [7]:
airfoil_section = AirfoilSection(airfoil, chord=console_chord)
wing_console = RectangularWingConsole(airfoil_section, length=console_length, materials=MATERIALS,
                                      min_length=min_console_length, max_length=max_console_length, 
                                      min_chord=min_chord_length, max_chord=max_chord_length,
                                      make_lattice=False)

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05


In [8]:
assy = cq.Assembly()
assy.add(wing_console.foam, name="foam", color=cq.Color("lightgray"))
assy.add(wing_console.front_box, name="left_box", color=cq.Color("yellow"))
assy.add(wing_console.central_box, name="central_box", color=cq.Color("yellow"))
assy.add(wing_console.rear_box, name="right_box", color=cq.Color("yellow"))
assy.add(wing_console.shell, name="shell", color=cq.Color("lightskyblue2"))
show(assy, angular_tolerance=0.1)

100% ⋮————————————————————————————————————————————————————————————⋮ (5/5)  0.89s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

## Find solution of the required wing length that satisfies the max wing tip displacement requirement

In [9]:
airfoil_section = AirfoilSection(airfoil, chord=150)
reynolds = airfoil_section.eval_reynolds(air_props)
reynolds

number of profile data points: 35
number of brep control points: 12


359589.0410958904

In [10]:
box_section = ThreeChamberBoxedWingSection(airfoil_section) #thickness_tol=0
wing_console = RectangularWingConsole(box_section, materials=MATERIALS, 
                                      min_length=min_console_length, max_length=max_console_length, 
                                      min_chord=min_chord_length, max_chord=max_chord_length,
                                      make_lattice=False)

new front wall position: 0.05


In [11]:
alpha_min_drag = wing_console.airfoil_section.airfoil.alpha_min_drag(reynolds)
alpha_min_drag

0.3604633440024395

In [12]:
solved_console_length = wing_console.fit_length_to_required_tip_deflection(
    alpha_min_drag, air_props, load_factor=load_factor
)

Finding wing console length for the chord = 150
Console length: 4614.493834438756 [mm]
Сonsole mass: 4.895838300423805 [kg]
Console spect ratio: 30.763292229591706
Lift force: 161.24994435891736 [N]
Absolute tip deflection: 14686.385385230064 [mm]
Relative tip deflection: 318.26644291131265 %
Box thickness: 1.427, [mm]
Tip deflection error: 3.082664429113126

Console length: 7435.506165561243 [mm]
Сonsole mass: 7.888847005707124 [kg]
Console spect ratio: 49.570041103741616
Lift force: 259.82805449407743 [N]
Absolute tip deflection: 99005.83535573375 [mm]
Relative tip deflection: 1331.527849634439 %
Box thickness: 1.427, [mm]
Tip deflection error: 13.215278496344391

Console length: 2871.0123311224866 [mm]
Сonsole mass: 3.0460571918911272 [kg]
Console spect ratio: 19.140082207483243
Lift force: 100.32532174865797 [N]
Absolute tip deflection: 2200.690007055977 [mm]
Relative tip deflection: 76.65205694869195 %
Box thickness: 1.427, [mm]
Tip deflection error: 0.6665205694869195

Console le

In [13]:
solved_console_length

1457.6691354990996

In [14]:
wing_console.stats(alpha_min_drag, air_props, load_factor=load_factor)

Length: 1457.6691354990996, [mm]
Chord: 150, [mm]
Aspect ratio: 9.717794236660664
Area: 0.21865037032486495, [m^2]
----------
Mass: 1.5465428362006948, [kg] (box: 0.9566951728256746, foam: 0.027119495312946544, shell: 0.5627281680620736)
Angle of attack: 0.3604633440024395, [degrees]
Excess lift force: 32.730145968005104, [N]
Console bend force: 35.771024681954856, [N]
Drag force: 1.320188563111719, [N]
Lift to weight ratio: 2.7983907782072066
Center of aerodynamic pressure (lift): (37.5, 728.8345677495498), [mm, mm]
----------
Reinforcement box thickness: 1.427, [mm]
Shell thickness: 1, [mm]
Bend stress: 26.02407584281799, [MPa]
Shear stress: 0.4433032656824103, [MPa]
Von Mises stress: 26.035400454551052, [MPa]
Safety factor: 11.138680217584565


## Find solution of the required wing console size that ensures sufficient lift force

In [15]:
required_console_lift_force = 0.5 * load_factor * payload_mass * g

In [16]:
solved_console_chord = wing_console.fit_chord_to_required_lift_force(
    alpha_min_drag, air_props, required_console_lift_force, load_factor=load_factor, tip_delta_max=0.1
)

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05
Finding wing console length for the chord = 412.86771068759987
Current chord: 412.86771068759987, [mm]
Console length: 6474.983245262038, [mm]
Console excess lift force: 94.84787859105234, [N]
Required lift force: 58.86, [N]
Console mass: 39.89886961035688, [kg]
Box thickness: 3.927, [mm]
Absolute error: 35.987878591052336, [N]

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05
Finding wing console length for the chord = 637.1322893124001
Current chord: 637.1322893124001, [mm]
Console length: 9342.096991599139, [mm]
Console excess lift force: -258.7647467444326, [N]
Required lift force: 58.86, [N]
Console mass: 128.83915785659798, [kg]
Box thickness: 6.06, [mm]
Absolute error: -317.6247467444326, [N]

number of profile data points: 35
number of brep control points: 12
new front wall position: 0.05
Finding wing console length for the chord = 2

In [17]:
wing_console.stats(alpha_min_drag, air_props, load_factor=load_factor)

Length: 2706.580666729388, [mm]
Chord: 236.7938424985267, [mm]
Aspect ratio: 11.430114221598597
Area: 0.6409016361070761, [m^2]
----------
Mass: 6.186342904534625, [kg] (box: 4.425559927814888, foam: 0.12548791748834967, shell: 1.6352950592313866)
Angle of attack: 0.3604633440024395, [degrees]
Excess lift force: 58.85999995129697, [N]
Console bend force: 71.01098866616412, [N]
Drag force: 3.009545597088431, [N]
Lift to weight ratio: 1.808508047113239
Center of aerodynamic pressure (lift): (59.19846062463167, 1353.290333364694), [mm, mm]
----------
Reinforcement box thickness: 2.252, [mm]
Shell thickness: 1, [mm]
Bend stress: 24.38853101346105, [MPa]
Shear stress: 0.35322905412599115, [MPa]
Von Mises stress: 24.396203747480627, [MPa]
Safety factor: 11.887095344903734


In [18]:
assy = cq.Assembly()
assy.add(wing_console.foam, name="foam", color=cq.Color("lightgray"))
assy.add(wing_console.front_box, name="left_box", color=cq.Color("yellow"))
assy.add(wing_console.central_box, name="central_box", color=cq.Color("yellow"))
assy.add(wing_console.rear_box, name="right_box", color=cq.Color("yellow"))
assy.add(wing_console.shell, name="shell", color=cq.Color("lightskyblue2"))
show(assy, angular_tolerance=0.1)

100% ⋮————————————————————————————————————————————————————————————⋮ (5/5)  4.41s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…